# ORIENTATION

###Needings
Still need to define a Scatterer with proper radius, wavelength, index of refraction, axis ratio and scattering geometry. The Scatterer 'orient' function is set by default to the function orientation.orient_single() to enable orientation averaging you need to set the 'orient' attribute to a function that is able to perform orientation averaging.

orientation package provide two function orient_averaged_fixed() and orient_averaged_adaptive(). I suggest you to use always the fixed version because it is much faster. 
You may want to define your own integration procedure ... it is possible.

Then you have to define the orientation pdf for the beta angle via the 'or_pdf' attribute. orientation module provides uniform_pdf() and gaussian_pdf(std). Beta is normal distributed as default with std=10.
As before you can define your own distribution (be careful about beta angle limits and normalization)
Alpha is always considered uniformly distributed.

Scatterer attributes n_alpha and n_beta define the number of integration points.

When orient attribute is set to an averaging function alpha and beta attributes are not considered.

In [ ]:
%pylab inline
from pytmatrix import tmatrix
from pytmatrix import tmatrix_aux as aux
from pytmatrix import refractive as ref
from pytmatrix import orientation as ori
from pytmatrix import scatter

eq_r = 10.0
scatterer = tmatrix.Scatterer(radius=eq_r, wavelength=aux.wl_C, m=ref.m_w_10C[aux.wl_C], 
                              axis_ratio=1.0/aux.dsr_thurai_2007(eq_r))

# ldr is a perfect quantity to check if particles are "oscillating"
print 'single alpha\t',scatterer.alpha, '\tbeta\t', scatterer.beta ,'\tldr\t', scatter.ldr(scatterer)

scatterer.orient = ori.orient_averaged_fixed
scatterer.or_pdf = ori.gaussian_pdf(std=20.0)
print 'ori avg alpha\t',scatterer.alpha, '\tbeta\t', scatterer.beta ,'\tldr\t', scatter.ldr(scatterer)

# try to change alpha and beta
scatterer.alpha = 60.0
scatterer.beta = 45.0
scatterer.orient = ori.orient_single # switch back to single orientation
print '\nsingle alpha\t',scatterer.alpha, '\tbeta\t', scatterer.beta ,'\tldr\t', scatter.ldr(scatterer)

scatterer.orient = ori.orient_averaged_fixed
print 'ori avg alpha\t',scatterer.alpha, '\tbeta\t', scatterer.beta ,'\tldr\t', scatter.ldr(scatterer)
print '\n Increase number of integration points'
scatterer.n_alpha = 10
scatterer.n_beta = 20
print 'ori avg alpha\t',scatterer.alpha, '\tbeta\t', scatterer.beta ,'\tldr\t', scatter.ldr(scatterer)


#PSD INTEGRATION

###Needings
To enable PSD integration you need to set the Scatterer attribute psd_integrator (default is None) psd.PSDintegrator object.
PSDintegrator objects holds some values used in the integration procedure and gives the opportunity to store and load scattering lookup tables. This is particurarly useful when you have recalculate radar properties over the same family of hydrometeors

Use the help command to get the docstring with the full list of attributes and methods

In [ ]:
from pytmatrix import psd
help(psd.PSDIntegrator)

## lookup tables
First we explore the aux functions ar(r)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

rad = np.linspace(1.0,5.0,100)
thurai = np.zeros(rad.shape)
dsr_bc = np.zeros(rad.shape)
dsr_pb = np.zeros(rad.shape)
for i in range(0,len(rad)):
    thurai[i] = aux.dsr_thurai_2007(rad[i])
    dsr_bc[i] = aux.dsr_bc(rad[i])
    dsr_pb[i] = aux.dsr_pb(rad[i])

plt.plot(rad,thurai)
plt.plot(rad,dsr_pb,'k')
plt.plot(rad,dsr_bc,'r')

Note that axis ratio values are less than 1, in fact these functions compute the ratio between the vertical and horizontal dimension, on the contrary the Scatterer object needs horizontal to vertical axis ratios.
We need to redefine axis ratio functions.

In [ ]:
scatterer.set_geometry(aux.geom_horiz_back)
print 'Single particle scattering', scatter.sca_intensity(scatterer) # we use this as reference for checking PSD integration
ar_fun = lambda D: 1.0/aux.dsr_thurai_2007(D)
# Ready to define a PSDIntegrator

from pytmatrix import psd
intPSD = psd.PSDIntegrator()
intPSD.D_max = 10.0 # this is maximum diameter for lookup table, but also the upper limit of the psd integration
intPSD.axis_ratio_func = ar_fun # if it is left to None, then ar is constant and equal to Scatterer.axis_ratio
intPSD.num_points = 1024 # increasing this number will increase the resolution of the lookup table in D
intPSD.geometries = (aux.geom_horiz_back,aux.geom_horiz_forw) # you need the lookup table for different geometries
                                                            # get different radar quantities
scatterer.psd_integrator = intPSD

PSD = psd.GammaPSD(D0=1.0, Nw=1e3, mu=4)
scatterer.psd = PSD

# even if you are not going to save it, you still need to init the scattering table
scatterer.psd_integrator.init_scatter_table(scatterer)
print 'PSD scattering', scatter.sca_intensity(scatterer)

# Change PSD parameters
scatterer.psd = psd.GammaPSD(D0=1.0, Nw=4e3, mu=4.0)
print 'PSD scattering', scatter.sca_intensity(scatterer)

scatterer.psd_integrator.save_scatter_table('./example_table','this is just an example table')

##NOTE
Scatterer objects use radius as principal dimension whereas PSDs uses diameter
The code handles the conversion, but you should be aware of that.

###Load lookup table

In [ ]:
particle = tmatrix.Scatterer()
particle.psd_integrator = psd.PSDIntegrator()
particle.psd_integrator.load_scatter_table('example_table')
particle.psd = psd.GammaPSD(D0=1.0,Nw=1e3,mu=4.0)

print scatter.sca_intensity(scatterer)
print scatter.sca_intensity(particle)


### First orient, then PSD integration
This will cause an error
If you ask for a scattering geometry not included in the lookup table an error will occur

In [ ]:
particle.beta=0.0
print 'beta ', particle.beta, 'Isca ', scatter.sca_intensity(particle)
particle.beta=10.0 # comment this line and run again this cell to continue
print 'beta ', particle.beta, 'Isca ', scatter.sca_intensity(particle)

### PSD integration hides any other option

In [ ]:
particle.radius=1.0
print 'radius\t ', particle.radius, '\tIsca\t ', scatter.sca_intensity(particle)
particle.radius=20.0
print 'radius\t ', particle.radius, '\tIsca\t ', scatter.sca_intensity(particle)
particle.wavelength=10.0
print 'wavelen\t ', particle.wavelength, '\tIsca\t ', scatter.sca_intensity(particle)
particle.wavelength=1.0
print 'wavelen\t ', particle.wavelength, '\tIsca\t ', scatter.sca_intensity(particle)

If PSDIntegrator is not set to None the code will use the S and Z matrix stored in the lookup table.
It is a much better idea to set up different configurations in different Scatterer objects.